# Tree segmentation with Sentinel 1/2 imagery - UNET Baseline

## John Brandt
## April 20, 2020

## Overview

This notebook contains the TensorFlow model training and prediction used to segment trees for [Restoration Mapper](https://restorationmapper.org). The notebook uses tensorflow 1.13.1 and additionally relies on Keras and tflearn. 

## Table of Contents

- Package loading
- Utility scripts
- Hyperparameter definitions
- Custom tensorflow layer functions
- Tensorflow graph creation
- Data loading
- Data preprocessing
- Equibatch creation
- Loss definition
- Tensorflow graph initialization
- Training
- Model validation
- Sanity Checks

## Package Loading

In [ ]:
from tqdm import tqdm_notebook, tnrange
import tensorflow as tf

sess = tf.Session()
from keras import backend as K
K.set_session(sess)

from time import sleep

import keras
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.layers import ELU
from keras.losses import binary_crossentropy
from tensorflow.python.ops import array_ops
from tensorflow.python.keras.layers import Conv2D, Lambda, Dense, Multiply, Add
from tensorflow.python.keras.activations import selu
from tensorflow.initializers import glorot_normal, lecun_normal
from scipy.ndimage import median_filter
from skimage.transform import resize

import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim import conv2d

import pandas as pd
import numpy as np
from random import shuffle
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
import itertools
from tflearn.layers.conv import global_avg_pool
from tensorflow.contrib.framework import arg_scope
from keras.regularizers import l1
from tensorflow.layers import batch_normalization
from tensorflow.python.util import deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Utility scripts

In [ ]:
%run ../../src/layers/zoneout.py
%run ../../src/layers/convgru.py
%run ../../src/layers/adabound.py
%run ../../src/layers/dropblock.py
%run ../../src/layers/extra_layers.py
%run ../../src/preprocessing/indices.py
%run ../../src/preprocessing/slope.py
%run ../../src/utils/metrics.py

# Hyperparameter definitions

In [ ]:
ZONE_OUT_PROB = 0.20
N_FILTERS = 32
AVERAGE_GRU_FILTERS = True
ACTIVATION_FUNCTION = 'relu'

INITIAL_LR = 1e-4
FINAL_LR = 2e-2
DROPBLOCK_MAXSIZE = 5
DECONV = 'nearest'
N_CONV_BLOCKS = 1
FINAL_ALPHA = 0.7
LABEL_SMOOTHING = 0.075
BATCH_RENORM = 'norm'

L2_REG = 0.0
BN_MOMENTUM = 0.9
BATCH_SIZE = 16
MAX_DROPBLOCK = 0.85

gru_flt = 32
fpa_flt = 28
out_conv_flt = 32

IMAGE_SIZE = 16
LABEL_SIZE = 14

# Hyperparameter grid search definitions

In [ ]:
# There are 3.8 million combinations of hyperparameters
params = {
    'out_conv_filter': range(16, 64, 16), # 5
    'activation_function': ['relu', 'selu', 'elu'], # 3
    'final_lr': [5e-3, 1e-2, 2e-2, 5e-2], # 4
    'dropblock_maxsize': [7, 6, 5, 4, 3], # 5
    'label_smoothing': [0., 0.025, 0.05, 0.075, 0.10, 0.125],
    'batch_renorm': ['renorm', 'norm']
}

# Custom layer definitions

### Utility blocks (Batch norm, cSSE, etc.)

In [ ]:
def Batch_Normalization(x, training, scope, clipping_params):
    '''Batch renormalization layer from tensorflow.layers

         Parameters:
          x (tf.Variable): input layer
          training (tf.Variable): flag to differentiate between train/test ops
          scope (str): tensorflow scope
          clipping_params (dict): specifies clipping of 
                                  rmax, dmax, rmin for renormalization

         Returns:
          x (tf.Variable): output of batch renormalization
          
         References:
          https://github.com/tensorflow/docs/blob/r1.13/site/en/
          api_docs/python/tf/layers/batch_normalization.md
    '''
    return batch_normalization(inputs=x, 
                               momentum = BN_MOMENTUM, 
                               training=training,
                               renorm = True,
                               reuse=None,
                               renorm_clipping = clipping_params,
                               name = scope)


def calc_renorm_params(epoch, n_samples, batch_size, k = 0):
    '''Calculates the clipping parameters for renormalization
       based on the learning schedule outlined in the original paper
       where rmax is initialized to 1, dmax to 0, rmin to 0,
       (effectively batch normalization), and then rmax and dmax
       are gradually reaxed to 3 and 5 over 40k and 25k steps,
       respectively. In this case, 40k has been reduced to 30k, and 25k
       has been reduced to 20k.

         Parameters:
          epoch (int): number of current training epoch
                       if testing, epoch number of model used
          n_samples (int): total number of training samples
          batch_size (int): training batch size 

         Returns:
          rmax (float)
          dmax (float)
          rmin (float)
    '''
    step = epoch * (n_samples // batch_size)
    step += k
    if step < 2500:
        rmax = 1.
        dmax = 0.
        rmin = 0.
    if step >= 2500:
        rmax = np.min([1 + 2*((step-2500)/25000), 3])
        dmax = np.min([1 + 5*((step-2500)/15000), 5])
        rmin = 0.
    return rmax, dmax, rmin

class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

### Conv blocks

In [ ]:
def conv_selu(inp, is_training, kernel_size, scope,
                filter_count = 16, pad = True, padding = 'valid', dilated = False,
                activation = True):
    '''Convolutional 2D layer with SELU activation and Lecun normal initialization
       with no batch norm. Only used if params['activation'] = 'selu'

         Parameters:
          inp (tf.Variable): (B, H, W, C) input layer
          is_training (str): flag to differentiate between train/test ops
          kernel_size (int): kernel size of convolution
          scope (str): tensorflow variable scope
          filter_count (int): number of convolution filters
          pad (bool): whether or not to reflect pad input
          padding (str): one of ['valid', 'same']
          dilated (bool): whether to perform atruous convolution
          activation (bool): whether to activate output

         Returns:
          conv (tf.Variable): output of Conv2D -> SELU
          
         References:
          https://arxiv.org/abs/1706.02515
    '''
    if activation:
        act = selu
    else:
        act = None
    if not dilated:
        padded = ReflectionPadding2D((1, 1,))(inp)
        conv = Conv2D(filters = filter_count, kernel_size = (kernel_size, kernel_size), activation = act,
                        padding = padding, kernel_initializer = lecun_normal())(padded)
    if not dilated and not pad:
        conv = Conv2D(filters = filter_count, kernel_size = (kernel_size, kernel_size), activation = act,
                        padding = padding, kernel_initializer = lecun_normal())(inp)
    if dilated:
        padded = ReflectionPadding2D((2, 2,))(inp)
        conv = Conv2D(filters = filter_count, kernel_size = (3, 3), activation = act, dilation_rate = (2, 2),
                        padding = padding, kernel_initializer = lecun_normal())(padded)
    return conv

def conv_bn_relu(inp, 
                 is_training, 
                 kernel_size,
                 scope,
                 filters, 
                 clipping_params,
                 keep_rate,
                 activation = True,
                 use_bias = False,
                 batch_norm = True):
    
    '''2D convolution, batch renorm, relu block, 3x3 drop block. 
       Use_bias must be set to False for batch normalization to work. 
       He normal initialization is used with batch normalization.
       RELU is better applied after the batch norm.
       DropBlock performs best when applied last, according to original paper.

         Parameters:
          inp (tf.Variable): input layer
          is_training (str): flag to differentiate between train/test ops
          kernel_size (int): size of convolution
          scope (str): tensorflow variable scope
          filters (int): number of filters for convolution
          clipping_params (dict): specifies clipping of 
                                  rmax, dmax, rmin for renormalization
          activation (bool): whether to apply RELU
          use_bias (str): whether to use bias. Should always be false

         Returns:
          bn (tf.Variable): output of Conv2D -> Batch Norm -> RELU
        
         References:
          http://papers.nips.cc/paper/8271-dropblock-a-regularization-
              method-for-convolutional-networks.pdf
          https://arxiv.org/abs/1702.03275
          
    '''
    
    conv = Conv2D(filters = filters, kernel_size = (kernel_size, kernel_size), 
                  activation = None, padding = 'valid', use_bias = use_bias,
                  kernel_initializer = tf.keras.initializers.he_normal())(inp)
    if batch_norm:
        conv = Batch_Normalization(conv, is_training, scope, clipping_params)
    if activation:
        conv = tf.nn.relu(conv)
    drop_block = DropBlock2D(keep_prob=keep_rate, block_size=3)
    conv = drop_block(conv, is_training)

    return conv

# Model definition

## Placeholders

In [ ]:
n_bands = 16*12
reg = keras.regularizers.l2(L2_REG) # for GRU
inp = tf.placeholder(tf.float32, shape=(None, IMAGE_SIZE, IMAGE_SIZE, n_bands))
labels = tf.placeholder(tf.float32, shape=(None, 14, 14))#, 1))
keep_rate = tf.placeholder_with_default(1.0, ()) # For DropBlock
is_training = tf.placeholder_with_default(False, (), 'is_training') # For BN, DropBlock
ft_lr = tf.placeholder_with_default(0.001, shape = ()) # For loss scheduling

In [ ]:
rmax = tf.placeholder(tf.float32, shape = ())
rmin = tf.placeholder(tf.float32, shape = ())
dmax = tf.placeholder(tf.float32, shape = ())

clipping_params = {
    'rmax': rmax,
    'rmin': rmin,
    'dmax': dmax
}

## Layers

In [ ]:
# DOWN 1
print("Input: {}".format(inp.shape))
x1 = ReflectionPadding2D((1, 1,))(inp)
conv1 = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "conv1", filters = N_FILTERS, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)
print("Conv1: {}".format(conv1.shape))



x1 = ReflectionPadding2D((1, 1,))(conv1)
x = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "conv2", filters = N_FILTERS, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)
print("Conv2: {}".format(x.shape))


down16 = tf.concat([x, conv1], axis = -1)
print(down16.shape)

maxpool1 = MaxPool2D(pool_size=(2, 2), strides=None)(down16)
print("Maxpool: {}".format(maxpool1.shape))


# DOWN 2
x1 = ReflectionPadding2D((1, 1,))(maxpool1)
x1 = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "conv9", filters = N_FILTERS*2, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)

x1 = ReflectionPadding2D((1, 1,))(x1)
x = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "conv10", filters = N_FILTERS*2, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)

down8 = tf.concat([x, maxpool1], axis = -1)
maxpool2 = MaxPool2D(pool_size=(2, 2), strides=None)(down8)
print("Maxpool 2: {}".format(maxpool2.shape))

# 4x4 Block
x1 = ReflectionPadding2D((1, 1,))(maxpool2)
x1 = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "conv13", filters = N_FILTERS*4, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)
x1 = ReflectionPadding2D((1, 1,))(x1)
print("Down mid: {}".format(x1.shape))

x = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "conv14", filters = N_FILTERS*4, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)
print("Down out: {}".format(x.shape))

# UPSAMPLING 4 - 8
three_up = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(x)
three_up = tf.concat([three_up, down8], axis = -1)

three_up = ReflectionPadding2D((1, 1,))(three_up)
three_up = conv_bn_relu(inp = three_up, is_training = is_training, 
           kernel_size = 3, scope =  'upconv1',
           filters = N_FILTERS*2, clipping_params = clipping_params,
           keep_rate = keep_rate, activation = True,
           use_bias = False, batch_norm = True)
print(three_up.shape)

x1 = ReflectionPadding2D((1, 1,))(three_up)
x = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "convup1", filters = N_FILTERS*2, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)

# Upsampling 8 - 16
three_up = tf.keras.layers.UpSampling2D((2, 2), interpolation = 'nearest')(three_up)
three_up = tf.concat([three_up, down16], axis = -1)
three_up = ReflectionPadding2D((1, 1,))(three_up)
three_up = conv_bn_relu(inp = three_up, is_training = is_training, 
           kernel_size = 3, scope =  'upconv2',
           filters = N_FILTERS, clipping_params = clipping_params,
           keep_rate = keep_rate, activation = True,
           use_bias = False, batch_norm = True)
print(three_up.shape)

x1 = ReflectionPadding2D((1, 1,))(three_up)
x = conv_bn_relu(inp = x1, 
                 is_training = is_training, kernel_size = 3,
                 scope = "convup2", filters = N_FILTERS, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)

# Out block
x = conv_bn_relu(inp = three_up, 
                 is_training = is_training, kernel_size = 3,
                 scope = "convout", filters = N_FILTERS, 
                 clipping_params = clipping_params,
                 activation = True, keep_rate = keep_rate, 
                 use_bias = False, batch_norm = True)
print(x.shape)

# 

print("Initializing last sigmoid bias with -2.94 constant")
init = tf.constant_initializer([-np.log(0.7/0.3)]) # For focal loss
fm = Conv2D(filters = 1,
            kernel_size = (1, 1), 
            padding = 'valid',
            activation = 'sigmoid',
            bias_initializer = init,
           )(x) # For focal loss

In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print("This model has {} parameters".format(total_parameters))

# Data loading

*  Load in CSV data from Collect Earth
*  Reconstruct the X, Y grid for the Y data per sample
*  Calculate NDVI, EVI, SAVI, BI, MSAVI2, and SI
*  Stack X, Y, length data
*  Apply median filter to DEM

In [ ]:
train_x = np.load("../../tile_data/processed/data_x_l2a_processed.npy")
train_y = np.load("../../tile_data/processed/data_y_l2a_processed.npy")

train_x = np.delete(train_x, 14, -1) 

# Data preprocessing

*  Identify and remove samples with time steps / channels that have a 0. or 1. value, which indicates missing data
*  Identify and remove samples with time steps / channels with no variation, which indicates missing data
*  Identify and remove samples with values above or below the allowable values for the band
*  Identify and remove samples with null data, or samples with extreme band 0 data (which squash all the "clean" samples)
*  Smooth per-pixel temporal data with Whittaker smoother, d = 2, lambda = 0.5 to reduce sample noise

In [ ]:
below_1 = [i for i, val in enumerate(train_x[:, :, :, :, :-2]) if np.min(val) < -1.5]
above_1 = [i for i, val in enumerate(train_x[:, :, :, :, :-2]) if np.max(val) > 1.5]
min_vals = [np.min(val) for i, val in enumerate(train_x[:, :, :, :, :-2]) if np.min(val) < -1.5]
max_vals = [np.max(val) for i, val in enumerate(train_x[:, :, :, :, :-2]) if np.max(val) > 1.5]
nans = [i for i, val in enumerate(train_x) if np.sum(np.isnan(val)) > 0]
oob_vals = [i for i, val in enumerate(train_x) if np.max(val[:, :, :, 0]) > 0.7]

outliers = below_1 + above_1 + nans + oob_vals
outliers = list(set(outliers))
print("Removing {} outlying training data points".format(len(outliers)))
print("\n")

train_x = train_x[[x for x in range(0, len(train_x)) if x not in outliers]]
train_y = train_y[[x for x in range(0, len(train_y)) if x not in outliers]]

## Min, Max scaling

In [ ]:
min_all = []
max_all = []

for band in range(0, train_x.shape[-1]):
    mins = np.percentile(train_x[:, :, :, :, band], 1)
    maxs = np.percentile(train_x[:, :, :, :, band], 99)

    train_x[:, :, :, :, band] = np.clip(train_x[:, :, :, :, band], mins, maxs)
    midrange = (maxs + mins) / 2
    rng = maxs - mins
    standardized = (train_x[:, :, :, :, band] - midrange) / (rng / 2)
    train_x[:, :, :, :, band] = standardized
    
    min_all.append(mins)
    max_all.append(maxs)
    
print("The data has been scaled to [{}, {}]".format(np.min(train_x), np.max(train_x)))

# Augment training data

Horizontal and vertical flips for 4x augmentation.

**To do**
*  Random guassian noise
*  Brightness, contrast
*  Region swaps (randomply position positive samples at different locations)

## Load and process test data

In [ ]:
test_x = np.load("../../tile_data/processed/test_x_l2a_processed.npy")
test_y = np.load("../../tile_data/processed/test_y_l2a_processed.npy")

test_x = np.delete(test_x, 14, -1)

below_1 = [i for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.min(val) < -1.67]
above_1 = [i for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.max(val) > 1.67]
min_vals = [np.min(val) for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.min(val) < -1.5]
max_vals = [np.max(val) for i, val in enumerate(test_x[:, :, :, :, :-2]) if np.max(val) > 1.5]
nans = [i for i, val in enumerate(test_x) if np.sum(np.isnan(val)) > 0]
outliers = below_1 + above_1 + nans
outliers = list(set(outliers))
print("There are {} outliers".format(len(outliers)))
print("\n")
print(min_vals, max_vals)
        
test_x = test_x[[x for x in range(0, len(test_x)) if x not in outliers]]
test_y = test_y[[x for x in range(0, len(test_y)) if x not in outliers]]

for sample in tnrange(0, len(test_x)):
    filtered = median_filter(test_x[sample, 0, :, :, 10], size = 5)
    filtered = np.reshape(filtered, (8, 2, 8, 2))
    filtered = np.mean(filtered, axis = (1, 3))
    filtered = resize(filtered, (16, 16), 0)
    test_x[sample, :, :, :, 10] = np.stack([filtered] * 24)
    
for band in range(0, test_x.shape[-1]):
    mins = min_all[band]
    maxs = max_all[band]
    test_x[:, :, :, :, band] = np.clip(test_x[:, :, :, :, band], mins, maxs)
    midrange = (maxs + mins) / 2
    rng = maxs - mins
    standardized = (test_x[:, :, :, :, band] - midrange) / (rng / 2)
    test_x[:, :, :, :, band] = standardized
    
    
print("The data has been scaled to [{}, {}]".format(np.min(test_x), np.max(test_x)))
print(test_x.shape)

In [ ]:
print("Train and test characteristics:")
print("Train mean Y {}".format(np.mean([np.sum(x) for x in test_y])))
print("Test STD Y {}".format(np.std([np.sum(x) for x in test_y])))
#print("Train number with zero trees {}".format(0.2*len([x for x in data_y[train_ids] if np.sum(x) == 0])))
#print("Test number with zero trees {}".format(0.8*len([x for x in data_y[test_ids] if np.sum(x) == 0])))
print("Train mean NDVI")
print("Test mean NDVI")
#print("There are {} train and {} test samples".format(len(train_ids), len(test_ids)))
#print("There is {} overlap between train and test".format(len([x for x in train_ids if x in test_ids])))

# Equibatch creation

The modelling approach uses equibatch sampling to ensure that there is a near constant standard deviation of the percent tree cover in the output labels for each batch. This helps ensure that the model performs equally well across gradients of tree cover, by mitigating the random possibility that many batches in a row near the end of sampling may be randomly biased towards a tree cover range.

In [ ]:
sums = np.sum(train_y, axis = (1, 2))
percents = [np.percentile(sums, x) for x in range(3*10, 100, 10)]
print(percents)
print("There are {} zeros".format(len(np.argwhere(sums == 0))))

In [ ]:
train_ids = [x for x in range(0, len(train_y))]

def multiplot(matrices):
    '''Plot multiple heatmaps with subplots
    
         Parameters:
          matrices (list of arrays):

         Returns:
          None
    '''
    fig, axs = plt.subplots(ncols=4)
    fig.set_size_inches(20, 4)
    for i, matrix in enumerate(matrices):
        sns.heatmap(data = matrix, ax = axs[i], vmin = 0, vmax = 0.9)
        axs[i].set_xlabel("")
        axs[i].set_ylabel("")
        axs[i].set_yticks([])
        axs[i].set_xticks([])

def equibatch(train_ids, p = percents, lovasz = False):
    '''Docstring
    
         Parameters:
          train_ids (list):
          p (list):
          lovasz (bool):

         Returns:
          equibatches (list):
    '''
    percents = [7.0, 13.0, 21.0, 33.0, 50.0, 80.0, 130.0] # overall
    np.random.shuffle(train_ids)
    ix = train_ids
    percs = [np.sum(x) for x in train_y[ix]]
    ids0 = [x for x, z in zip(ix, percs) if z <= 2]
    ids30 = [x for x, z in zip(ix, percs) if 2 < z < percents[0]]
    ids40 = [x for x, z in zip(ix, percs) if percents[0] < z < percents[1]]
    ids50 = [x for x, z in zip(ix, percs) if percents[1] < z < percents[2]]
    ids60 = [x for x, z in zip(ix, percs) if percents[2] < z < percents[3]]
    ids70 = [x for x, z in zip(ix, percs) if percents[3] < z < percents[4]]
    ids80 = [x for x, z in zip(ix, percs) if percents[4] < z < percents[5]]
    ids90 = [x for x, z in zip(ix, percs) if percents[5] < z < percents[6]]
    ids100 = [x for x, z in zip(ix, percs) if percents[6] < z]
    

    new_batches = []
    maxes = [len(ids0), len(ids30), len(ids40), len(ids50), len(ids60), len(ids70),
             len(ids80), len(ids90), len(ids100)]
    print(maxes)
    cur_ids = [0] * len(maxes)
    iter_len = len(train_ids)//(len(maxes))
    for i in range(0, iter_len):
        for i, val in enumerate(cur_ids):
            if val > maxes[i] - 1:
                cur_ids[i] = 0
        if cur_ids[0] >= (maxes[0] - 2):
            cur_ids[0] = 0
        to_append = [ids0[cur_ids[0]],
                    ids30[cur_ids[1]], ids40[cur_ids[2]],
                    ids50[cur_ids[3]],
                    ids60[cur_ids[4]], ids70[cur_ids[5]], ids80[cur_ids[6]],
                    ids90[cur_ids[7]], ids100[cur_ids[8]]]
        
        np.random.shuffle(to_append)
        new_batches.append(to_append)
        cur_ids = [x + 1 for x in cur_ids]
        
    new_batches = [item for sublist in new_batches for item in sublist]
    return new_batches

batch = equibatch(train_ids)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
f.set_size_inches(15, 6)
sns.distplot(np.sum(train_y, axis = (1, 2)), bins = 100, kde = False, ax = ax1)
ax1.set_title('Original distribution')
ax2.set_title('Equibatch distribution')
sns.distplot(np.sum(train_y[batch], axis = (1, 2)),
             bins = 100, kde = False, ax = ax2)

plt.show()

## Weight cross entropy by effective number of samples

In [ ]:
sum_pos = np.sum(train_y[batch])
sum_neg = len(train_y[batch]) * 196 - sum_pos
beta = (sum_pos + sum_neg - 1) / (sum_pos + sum_neg)
print("Beta: {}".format(beta))
samples_per_cls = np.array([sum_neg, sum_pos])
effective_num = 1.0 - np.power(beta, samples_per_cls)
print(effective_num)
weights = (1.0 - beta) / np.array(effective_num)
weights = weights / np.sum(weights)
print("Neg and pos weights: {}".format(weights))
weight = weights[1] / weights[0]
print(weight)


In [ ]:
print("Baseline: The positive is: {}".format(weights[0]))
print("Baseline: The negative is: {}".format(weights[1]))
print("\n")
print("Balanced: The positive is: {}".format(weight*weights[0]))
print("Balanced: The negative is: {}".format(weights[1]))


# Loss definition

The current best loss is a combination of weighted binary cross entropy and per-image Lovasz-Softmax, with a loss schedule with the latter becoming more important each epoch

In [ ]:
import math

def weighted_bce_loss(y_true, y_pred, weight, mask = None, smooth = 0.025):
    '''Calculates the weighted binary cross entropy loss between y_true and
       y_pred with optional masking and smoothing for regularization
    
         Parameters:
          y_true (arr):
          y_pred (arr):
          weight (float):
          mask (arr):
          smooth (float):

         Returns:
          loss (float):
    '''
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    y_true = K.clip(y_true, smooth, 1. - smooth)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = tf.nn.weighted_cross_entropy_with_logits(
        y_true,
        logit_y_pred,
        weight,
    )
    return tf.reduce_mean(loss)

# Evaluation metrics

In [ ]:
countries = {'africaeast-test': [0, 34],
 'africawest-test': [35, 96],
 'cameroon-test': [97, 135],
 'ethiopia-test': [136, 248],
 'ghana-test': [249, 282],
 'ghana-test-large': [283, 318],
 'global-test': [319, 460],
 'india-test': [461, 511],
 'kenya-test': [512, 600],
 'lac-north-test': [601, 641],
 'lac-south-test': [642, 683], 'all': [0, 683]}

def dice_loss(y_true, y_pred):
    '''Calculates the dice score of ground truth and prediction arrays
    
         Parameters:
          y_true (arr):
          y_pred (arr):
          
         Returns:
          metric (arr):
    '''
    numerator = 2 * np.sum(y_true * y_pred, axis=-1)
    denominator = np.sum(y_true + y_pred, axis=-1)
    return (numerator + 1) / (denominator + 1)

def compute_f1_score_at_tolerance(true, pred, tolerance = 1):
    fp = 0
    tp = 0
    fn = 0
    
    tp = np.zeros_like(true)
    fp = np.zeros_like(true)
    fn = np.zeros_like(true)
    
    
    for x in range(true.shape[0]):
        for y in range(true.shape[1]):
            min_x = np.max([0, x-1])
            min_y = np.max([0, y-1])
            max_y = np.min([true.shape[0], y+2])
            max_x = np.min([true.shape[0], x+2])
            if true[x, y] == 1:
                if np.sum(pred[min_x:max_x, min_y:max_y]) > 0:
                    tp[x, y] = 1
                else:
                    fn[x, y] = 1
            if pred[x, y] == 1:
                if np.sum(true[min_x:max_x, min_y:max_y]) > 0:
                    if true[x, y] == 1:
                        tp[x, y] = 1
                else:
                    fp[x, y] = 1                
                
    precision =  np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * ((precision * recall) / (precision + recall))
    return np.sum(tp), np.sum(fp), np.sum(fn)

def calculate_metrics(country, al = 0.4, canopy_thresh = 100):
    '''Calculates the following metrics for an input country, based on
       indexing of the country dictionary:
       
         - Loss
         - F1
         - Precision
         - Recall
         - Dice
         - Mean surface distance
         - Average error
    
         Parameters:
          country (str):
          al (float):
          
         Returns:
          val_loss (float):
          best_dice (float):
          error (float):
    '''
    print(canopy_thresh)
    start_idx = countries[country][0]
    stop_idx = countries[country][1]
    best_f1 = 0
    best_dice = 0
    best_thresh = 0
    p = 0
    r = 0
    error = 0
    d = 0
    ys = []
    vls = []
    t_alls = []
    test_ids = [x for x in range(len(test_x))]
    for test_sample in test_ids[start_idx:stop_idx]:
        if np.sum(test_y[test_sample]) < ((canopy_thresh/100) * 197) :
            y, vl = sess.run([fm, test_loss], feed_dict={inp: test_x[test_sample].reshape(1, 16, 16, n_bands),
                                              is_training: False,
                                              clipping_params['rmax']: rmax_epoch,
                                              clipping_params['rmin']: rmin_epoch,
                                              clipping_params['dmax']: dmax_epoch,
                                              labels: test_y[test_sample, :, :].reshape(1, 14, 14),
                                              })
            ys.append(y.reshape((14, 14)))
            vls.append(vl)
            t = test_y[test_sample].reshape((14, 14))
            t_alls.append(t)

            
    for thresh in range(10, 11):
        all_preds, all_trues = [], []
        tps, fps, fns = [], [], []
        abs_error, haus = [], []
        trues, preds = [], []
        dice_losses = []
        val_loss = []
        for sample in range(len(ys)):
            pred = np.copy(ys[sample])
            true = t_alls[sample]
            vl = vls[sample]
            pred[np.where(pred >= thresh*0.05)] = 1
            pred[np.where(pred < thresh*0.05)] = 0
            
            true_s = np.sum(true[1:-1])
            pred_s = np.sum(pred[1:-1])
            abs_error.append(abs(pred_s - true_s))
            tp, fp, fn = compute_f1_score_at_tolerance(true, pred)
            tps.append(tp)
            fps.append(fp)
            fns.append(fn)
            trues.append(true_s)
            preds.append(pred_s)
            all_trues.append(true.flatten())
            all_preds.append(pred.flatten())
            val_loss.append(np.mean(vl))
            
        oa_error = np.mean(abs_error)
        precision = np.sum(tps) / (np.sum(tps) + np.sum(fps))
        recall = np.sum(tps) / (np.sum(tps) + np.sum(fns))
        f1 = 2*((precision* recall) / (precision + recall))

        if f1 > best_f1:
            best_f1 = f1
            p = precision
            r = recall
            error = oa_error
            best_thresh = thresh*0.05
            preds_return = all_preds
            trues_return = all_trues
    print("{}: Val loss: {} Thresh: {} F1: {}"
          " R: {} P: {} Error: {}".format(country, 
                                         np.around(np.mean(val_loss), 3),
                                         np.around(best_thresh, 2),
                                         np.around(best_f1, 3), np.around(p, 3), np.around(r, 3), 
                                         np.around(error, 3)))
    return np.mean(val_loss), best_f1, error, preds_return, trues_return

# Graph set up

In [ ]:
def augment_batch(batch_ids, batch_size):
    '''Performs random flips and rotations of the X and Y
       data for a total of 4 x augmentation
    
         Parameters:
          batch_ids (list):
          batch_size (int):
          
         Returns:
          x_batch (arr):
          y_batch (arr):
    '''
    x = train_x[batch_ids, :, :, :]
    y = train_y[batch_ids, :, :]
    x_batch = np.zeros_like(x)
    y_batch = np.zeros_like(y)
    flips = np.random.choice(np.array([0, 1, 2, 3]), batch_size, replace = True)
    for i in range(x.shape[0]):
        current_flip = flips[i]
        if current_flip == 0:
            x_batch[i, :, :, :] = x[i]
            y_batch[i, :, :] = y[i]
        if current_flip == 1:
            x_batch[i, :, :, :] = np.flip(x[i], 1)
            y_batch[i, :, :] = np.flip(y[i], 0)
        if current_flip == 2:
            x_batch[i, :, :, :] = np.flip(x[i], [2, 1])
            y_batch[i, :, :] = np.flip(y[i], [1, 0])
        if current_flip == 3:
            x_batch[i, :, :, :] = np.flip(x[i], 2)
            y_batch[i, :, :] = np.flip(y[i], 1)

    y_batch = y_batch.reshape((batch_size, 14, 14))
    return x_batch, y_batch

x_batch_test, y_batch_test = augment_batch([0, 1], 2)


In [ ]:
FRESH_START = True
best_val = 0.2

START_EPOCH = 1
END_EPOCH = 150
LEARNING_RATE = 3e-3
test_ids = [x for x in range(0, len(test_x))]

print("Starting model with: \n {} zone out \n {} l2 \n {} initial LR \n {} final LR \n {} parameters"
     .format(ZONE_OUT_PROB, L2_REG, INITIAL_LR, FINAL_LR, total_parameters))

if not FRESH_START:
    print("Resuming training with a best validation score of {}".format(best_val))
    
if FRESH_START:
    print("Restarting training from scratch on {} "
          "train and {} test samples, total {}".format(len(train_ids), len(test_ids), len(train_ids)/4))

    # current best 1e-4 5e-3
    optimizer = AdaBoundOptimizer(1e-3, 1e-1)
    train_loss = weighted_bce_loss(tf.reshape(labels, (-1, 14, 14, 1)), fm, weight = weight, smooth = 0.)
    #l2_loss = tf.losses.get_regularization_l05oss()
    #train_loss += l2_loss

    ft_optimizer = tf.train.GradientDescentOptimizer(ft_lr)
    
    test_loss = weighted_bce_loss(tf.reshape(labels, (-1, 14, 14, 1)), fm, weight = 2.36, smooth = 0.)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(train_loss)   
        ft_op = ft_optimizer.minimize(train_loss)
        
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    saver = tf.train.Saver(max_to_keep = 75)
    
print("The graph has been finalized")

In [ ]:
# This cell should be run to do fine-tuning, if commented - train from scratch
path = "../../models/unet-resnet/new/34-65-9/"
new_saver = tf.train.import_meta_graph(path + 'model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint(path))

In [ ]:
train_losses = []
test_losses = []
f1s = []
errors = []

# Mean

In [ ]:
train_x = np.mean(train_x, axis = 1)
test_x = np.mean(test_x, axis = 1)


# Median

In [ ]:
train_x = np.median(train_x, axis = 1)
test_x = np.median(test_x, axis = 1)

# Median + S.D.

In [ ]:
mean_x = np.median(train_x, axis = 1)
std_x = np.std(train_x, axis = 1)
train_x = np.concatenate([mean_x, std_x], axis = -1)

In [ ]:
mean_x = np.median(test_x, axis = 1)
std_x = np.std(test_x, axis = 1)
test_x = np.concatenate([mean_x, std_x], axis = -1)

# Quarterly means

In [ ]:
train_x = np.reshape(train_x, (train_x.shape[0], 4, 6, 16, 16, train_x.shape[-1]))
train_x = np.mean(train_x, axis = 2)
train_x = np.swapaxes(train_x, 1, 3)
train_x = np.swapaxes(train_x, 1, 2)
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 
                               train_x.shape[2], train_x.shape[-1]*train_x.shape[-2]))

In [ ]:
test_x = np.reshape(test_x, (test_x.shape[0], 4, 6, 16, 16, test_x.shape[-1]))
test_x = np.mean(test_x, axis = 2)
test_x = np.swapaxes(test_x, 1, 3)
test_x = np.swapaxes(test_x, 1, 2)
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 
                               test_x.shape[2], test_x.shape[-1]*test_x.shape[-2]))

# Monthly means

In [ ]:
train_x = np.reshape(train_x, (train_x.shape[0], 12, 2, 16, 16, train_x.shape[-1]))
train_x = np.mean(train_x, axis = 2)
train_x = np.swapaxes(train_x, 1, 3)
train_x = np.swapaxes(train_x, 1, 2)
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 
                               train_x.shape[2], train_x.shape[-1]*train_x.shape[-2]))

test_x = np.reshape(test_x, (test_x.shape[0], 12, 2, 16, 16, test_x.shape[-1]))
test_x = np.mean(test_x, axis = 2)
test_x = np.swapaxes(test_x, 1, 3)
test_x = np.swapaxes(test_x, 1, 2)
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 
                               test_x.shape[2], test_x.shape[-1]*test_x.shape[-2]))

In [ ]:
import scipy
best_val = 0.5
fine_tune = False
countries['all'] = [0, len(test_x)]
ft_epochs = 0
ft_learning_rate = 1e-2

train_ids = [x for x in range(len(train_y))]
for i in range(1, 40):
    if fine_tune == True:
        op = ft_op
        ft_epochs += 1
        if ft_epochs % 15 == 0:
            ft_learning_rate /= 2
        print("FINE TUNING WITH {} LR".format(ft_learning_rate))
        
    else:
        op = op = train_op
    randomize = equibatch(train_ids)
    print("starting epoch {}, drop: {}".format(i, np.max((1. - (i*0.003), 0.75))))
    
    loss = train_loss
    BATCH_SIZE = 20
    test_ids = [x for x in range(0, len(test_x))]
    losses = []
    
    for k in tnrange(int(len(randomize) // BATCH_SIZE)):
        rmax_epoch, dmax_epoch, rmin_epoch = calc_renorm_params(i, len(train_y), 20, k)
        if k % 40 == 0:
            sleep(1)
        batch_ids = randomize[k*BATCH_SIZE:(k+1)*BATCH_SIZE]
        x_batch, y_batch = augment_batch(batch_ids, BATCH_SIZE)
        opt, tr = sess.run([op, loss],
                          feed_dict={inp: x_batch,
                                     labels: y_batch,
                                     is_training: True,
                                     clipping_params['rmax']: rmax_epoch,
                                     clipping_params['rmin']: rmin_epoch,
                                     clipping_params['dmax']: dmax_epoch,
                                     keep_rate: np.max((1. - (i*0.003), 0.9)),
                                     ft_lr: ft_learning_rate,
                                     })
        losses.append(tr)
    
    print("Epoch {}: Loss {}".format(i, np.around(np.mean(losses[:-1]), 3)))
    #_, _, _, _ = calculate_metrics("global-test", al = al)
    val_loss, f1, error, _, _ = calculate_metrics('all')
    train_losses.append(np.mean(losses[:-1]))
    test_losses.append(val_loss)
    f1s.append(f1)
    errors.append(error)
    if f1 > (best_val - 0.005):
        print("Saving model with {}".format(f1))
        os.mkdir("../../models/unet-resnet/monthly/{}-{}-{}/".format(str(i), str(f1*100)[:2], str(f1*100)[3]))
        save_path = saver.save(sess, "../../models/unet-resnet/monthly/{}-{}-{}/model".format(str(i), str(f1*100)[:2], str(f1*100)[3]))
        if f1 > best_val:
            best_val = f1

In [ ]:
val_loss, f1, error, _, _ = calculate_metrics('all')

# Model validation and sanity checks





In [ ]:
countries['all'] = [0, len(test_x)]
_, _, error, preds, trues = calculate_metrics('all')

In [ ]:
preds2 = np.concatenate(preds).flatten()
trues2 = np.concatenate(trues).flatten()
tp = preds2 * trues2
fn = [1 if x > y else 0 for (x, y) in zip(trues2, preds2)]
fp = [1 if y > x else 0 for (x, y) in zip(trues2, preds2)]
tn = (len(test_y) * 196) - np.sum(tp) - np.sum(fn) - np.sum(fp)
print("TP {}, FN {}, FP {}, TN {}".format(np.sum(tp), sum(fn), np.sum(fp), tn))
print(len(preds2))

tps = []
fns = []
fps = []
for i in range(0, len(test_y)*196, 196):
    tps.append(np.sum(tp[i:i+196]))
    fns.append(np.sum(fn[i:i+196]))
    fps.append(np.sum(fp[i:i+196]))
    
    

from sklearn.metrics import f1_score
true_canopy = []
error_canopy = []
pred_canopy = []
f1_hard = []
tp_softs = []
fp_softs = []
fn_softs = []
for i in range(len(trues)):
    true_canopy.append(np.sum(trues[i]) / 1.96)
    error_canopy.append(abs(np.sum(preds[i]) - np.sum(trues[i])) / 1.96)
    pred_canopy.append(np.sum(preds[i]) / 1.96)
    f1_hard.append(f1_score(trues[i], preds[i]))
    tp_soft, fp_soft, fn_soft = compute_f1_score_at_tolerance(np.array(trues[i].reshape((14, 14))),
                                                 np.array(preds[i].reshape((14, 14))))
    tp_softs.append(tp_soft)
    fp_softs.append(fp_soft)
    fn_softs.append(fn_soft)

In [ ]:
metrics = pd.DataFrame({'true': true_canopy,
                        'pred': pred_canopy,
                        'f1_hard': f1_hard,
                        'error': error_canopy,
                        'tp': tps,
                        'fp': fps,
                        'fn': fns,
                        'tp_soft': tp_softs,
                        'fp_soft': fp_softs,
                        'fn_soft': fn_softs,
                       })

res = map(lambda x: int(math.floor(np.min([x, 90]) / 10.0)) * 10, true_canopy)
res = [x for x in res]
metrics['group'] = res
metrics['model'] = 'U-net'

#etrics.to_csv("../../data/metrics/unet-sample.csv")

In [ ]:
precisions = (np.sum(metrics.tp_soft) / np.sum(metrics.tp_soft + metrics.fp_soft))

In [ ]:
recalls = (np.sum(metrics.tp_soft) / np.sum(metrics.tp_soft + metrics.fn_soft))
recalls

In [ ]:
hard_precisions = metrics.groupby('group').apply(lambda x: (np.sum(x.tp) / np.sum(x.tp + x.fp)))
hard_recalls = metrics.groupby('group').apply(lambda x: (np.sum(x.tp) / np.sum(x.tp + x.fn)))
errors = metrics.groupby('group').apply(lambda x: np.mean(x.error))
hard_f1 = 2 *  ((hard_precisions * hard_recalls) / (hard_precisions + hard_recalls))

precisions = metrics.groupby('group').apply(lambda x: (np.sum(x.tp_soft) / np.sum(x.tp_soft + x.fp_soft)))
recalls = metrics.groupby('group').apply(lambda x: (np.sum(x.tp_soft) / np.sum(x.tp_soft + x.fn_soft)))
soft_f1 = 2 *  ((precisions * recalls) / (precisions + recalls))


In [ ]:
new_metrics = pd.DataFrame({'group': [x for x in range(0, 100, 10)],
                            'hard_rec': hard_recalls,
                            'soft_rec': recalls,
                            'hard_prec': hard_precisions,
                            'soft_prec': precisions,
                            'hard_f1': hard_f1,
                            'soft_f1': soft_f1,
                            'error': errors,
                            'model': 'U-net'
                           })

new_metrics.to_csv("../../data/metrics/unet.csv")

In [ ]:
new_metrics

# Model results plotting

In [ ]:
start = 400

test_ids = [x for x in range(0, len(test_x))]

def multiplot(matrices, nrows = 2, ncols = 4):
    '''Docstring
    
         Parameters:
          matrices (list):
          nrows (int):
          
         Returns:
          None
    '''
    fig, axs = plt.subplots(ncols=4, nrows = nrows)
    fig.set_size_inches(20, 4*nrows)
    to_iter = [[x for x in range(i, i + ncols + 1)] for i in range(0, nrows*ncols, ncols)]
    for r in range(1, nrows + 1):
        min_i = min(to_iter[r-1])
        max_i = max(to_iter[r-1])
        for i, matrix in enumerate(matrices[min_i:max_i]):
            sns.heatmap(data = matrix, ax = axs[r - 1, i], vmin = 0, vmax = 0.9)
            axs[r - 1, i].set_xlabel("")
            axs[r - 1, i].set_ylabel("")
            axs[r - 1, i].set_yticks([])
            axs[r - 1, i].set_xticks([])
    plt.show



In [ ]:
from sklearn.metrics import f1_score
print(start/len(test_ids))
test_ids = sorted(test_ids)
matrix_ids = [test_ids[start], test_ids[start + 1], test_ids[start + 2], test_ids[start + 3],
              test_ids[start + 4], test_ids[start + 5], test_ids[start + 6], test_ids[start + 7]]
preds = []
trues = []
print(matrix_ids)
for i in matrix_ids:
    idx = i
    y = sess.run([fm], feed_dict={inp: test_x[idx].reshape(1, IMAGE_SIZE, IMAGE_SIZE, n_bands),
                                  length: test_lengths[idx].reshape(1, 1),
                                  is_training: False,
                                  clipping_params['rmax']: rmax_epoch,
                                  clipping_params['rmin']: rmin_epoch,
                                  clipping_params['dmax']: dmax_epoch,
                                  })
    y = np.array(y).reshape(14, 14)
    preds.append(y)
    y2 = np.copy(y)
    #y[np.where(y > 0.4)] = 1.0
    #y[np.where(y < 0.4)] = 0.
    #y[np.where(y >= 0.4)] = 1.
    #dc = np.around(dice_loss(test_y[idx].flatten(), y2.flatten()), 3)
    true = test_y[idx].reshape(14, 14)
    #y[np.where(y > 0.4)] = 1.
    identify_alignment(true, y)
    #f1 = f1_score(true.flatten(), y.flatten())
    #print(i, f1, (1-f1)*np.sum(true))
    #if np.sum(true[1:-1]) > 0 and np.sum(y[1:-1]) > 0:
    #    print(i, assd(true[1:-1], y[1:-1]))
    #print(i, np.sum(abs(true - y)))
    trues.append(true)
    
    
""


to_plot = trues[0:4] + preds[0:4] + trues[4:] + preds[4:]

multiplot(to_plot, nrows = 4, ncols = 4)
# 448"109, ""
start = start + 8 
# 50, 64, 66, 83, 94


In [ ]:
train_ids = [x for x in range(train_x.shape[0])]
train_ids = sorted(train_ids)
start = 25

In [ ]:
#atrix_ids = random.sample(755, 4)
matrix_ids = [train_ids[start], train_ids[start + 1], train_ids[start + 2], train_ids[start + 3], train_ids[start + 4],
             train_ids[start + 5], train_ids[start + 6], train_ids[start + 7], train_ids[start + 8]]

preds = []
trues = []
print(start//4)
print(matrix_ids)
for i in matrix_ids:
    idx = i
    y = sess.run([fm], feed_dict={inp: train_x[idx].reshape(1, IMAGE_SIZE, IMAGE_SIZE, n_bands),
                                  length: train_l[idx].reshape(1, 1),
                                  is_training: False,
                                  clipping_params['rmax']: rmax_epoch,
                                  clipping_params['rmin']: rmin_epoch,
                                  clipping_params['dmax']: dmax_epoch,
                                  })
    y = np.array(y).reshape(14, 14)
    #y[np.where(y > 0.3)] = 0.85
    preds.append(y)
    true = train_y[idx].reshape(14, 14)
    #identify_alignment(true, y)
    trues.append(true)
    
start += 8

to_plot = trues[0:4] + preds[0:4] + trues[5:] + preds[5:]
multiplot(to_plot, nrows = 4, ncols = 4)  